In [28]:
import requests
import pandas as pd
import time
import numpy as np
from datetime import timedelta
import math

In [29]:
dates = [['2020', '2021'], ['2021', '2022'], ['2022', '2023']]
months = [['05', '11'], ['12', '04']]
currencies = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'USDCNY', 'USDCHF']
fx_df = df = pd.DataFrame(columns=[])


In [30]:
for drange in dates:
    for cur in currencies:
        time.sleep(0.25)
        url = "https://api.polygon.io/v2/aggs/ticker/C:{}/range/1/minute/{}-05-11/{}-11-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq".format(
            cur, drange[0], drange[0])
        print(url)
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        json_data = response.json()
        relevant_data = json_data['results']
        tmp_df = pd.DataFrame(relevant_data)
        tmp_df['datetime'] = pd.to_datetime(tmp_df['t'], unit='ms')
        tmp_df['Currency'] = cur
        fx_df = pd.concat([fx_df, tmp_df], ignore_index=True)

        time.sleep(0.25)
        url = "https://api.polygon.io/v2/aggs/ticker/C:{}/range/1/minute/{}-12-11/{}-04-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq".format(
            cur, drange[0], drange[1])
        print(url)
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        json_data = response.json()
        relevant_data = json_data['results']
        tmp_df = pd.DataFrame(relevant_data)
        tmp_df['datetime'] = pd.to_datetime(tmp_df['t'], unit='ms')
        tmp_df['Currency'] = cur
        fx_df = pd.concat([fx_df, tmp_df], ignore_index=True)


https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/1/minute/2020-05-11/2020-11-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/1/minute/2020-12-11/2021-04-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:USDJPY/range/1/minute/2020-05-11/2020-11-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:USDJPY/range/1/minute/2020-12-11/2021-04-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:GBPUSD/range/1/minute/2020-05-11/2020-11-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:GBPUSD/range/1/minute/2020-12-11/2021-04-11?adjusted=true&sort=asc&limit=99999&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq
https://api.polygon.io/v2/aggs/ticker/C:AUDUSD

In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load your DataFrame with the relevant columns
df = fx_df  # Replace with the path to your DataFrame

# Drop rows with NaN values
df.dropna(inplace=True)

# Define the features and target variables
features = ['v', 'vw', 'o', 'c', 'h', 'l']

# Initialize the random forest regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a new DataFrame to store the predictions
predictions_df = pd.DataFrame(columns=['Currency', 'Long Probability', 'Short Probability'])

# Iterate over unique currencies
for currency in df['Currency'].unique():
    # Filter the data for the current currency
    currency_df = df[df['Currency'] == currency]

    # Calculate the price differences
    currency_df['price_diff'] = currency_df['c'].diff().shift(-1)

    # Drop the last row which will have NaN price difference
    currency_df.dropna(inplace=True)

    # Split the data into features and target
    X = currency_df[features]
    y = currency_df['price_diff']

    # Fit the regressor to the data
    regressor.fit(X, y)

    # Predict the next 24 hours price difference
    next_24_hours = df[df['Currency'] == currency].tail(24)[features]
    predictions = regressor.predict(next_24_hours)

    # Calculate probabilities
    long_probs = len(predictions[predictions > 0]) / len(predictions)
    short_probs = len(predictions[predictions < 0]) / len(predictions)

    # Add the predictions to the DataFrame
    predictions_df = predictions_df.append({
        'Currency': currency,
        'Long Probability': long_probs,
        'Short Probability': short_probs
    }, ignore_index=True)

# Display the predictions
print(predictions_df)


/var/folders/k1/m5zjfxfs7w3c62s26f_0vxbw0000gn/T/ipykernel_44109/543863497.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_df['price_diff'] = currency_df['c'].diff().shift(-1)
/var/folders/k1/m5zjfxfs7w3c62s26f_0vxbw0000gn/T/ipykernel_44109/543863497.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_df.dropna(inplace=True)
/var/folders/k1/m5zjfxfs7w3c62s26f_0vxbw0000gn/T/ipykernel_44109/543863497.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  p

  Currency  Long Probability  Short Probability
0   EURUSD          0.416667           0.583333
1   USDJPY          0.541667           0.458333
2   GBPUSD          0.458333           0.541667
3   AUDUSD          0.416667           0.583333
4   USDCAD          0.416667           0.583333
5   USDCNY          0.416667           0.583333
6   USDCHF          0.625000           0.375000


/var/folders/k1/m5zjfxfs7w3c62s26f_0vxbw0000gn/T/ipykernel_44109/543863497.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({


In [34]:
predictions_df['diff'] = predictions_df['Long Probability'] - predictions_df['Short Probability']

In [35]:
predictions_df


,Currency,Long Probability,Short Probability,diff
0,EURUSD,0.416667,0.583333,-0.166667
1,USDJPY,0.541667,0.458333,0.083333
2,GBPUSD,0.458333,0.541667,-0.083333
3,AUDUSD,0.416667,0.583333,-0.166667
4,USDCAD,0.416667,0.583333,-0.166667
5,USDCNY,0.416667,0.583333,-0.166667
6,USDCHF,0.625000,0.375000,0.250000
